# Using our pre-trained Tire recognition network

 

In [0]:
import os
!git clone https://github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')
os.chdir('samples')
!pip install imgaug
!pip install cython
!pip install pycocotools

In [0]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config


!wget https://s3.amazonaws.com/italia18/dataset_segmentation.zip
!unzip -o dataset_segmentation


sys.path.append(os.path.join(ROOT_DIR, "samples/dataset_segmentation/"))  # To find local version
import segmentation

%matplotlib inline 


# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_segmentation_0030.h5")
# Download COCO trained weights from Releases if needed
!wget https://s3.amazonaws.com/italia18/mask_rcnn_segmentation_0030.h5

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [0]:
class InferenceConfig(segmentation.SegmentationConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Create Model and Load Trained Weights

In [0]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights('/content/Mask_RCNN/samples/mask_rcnn_segmentation_0030.h5', by_name=True)

In [0]:
!pwd

## Class Names

Here we just set a list with the possible class outcomes

In [0]:
class_names = ['Background', 'Tire']

## Run Object Detection

In [0]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread('https://i.pinimg.com/564x/82/b6/11/82b611cb6360cf47ecb7774ba9f80b7a.jpg')

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])